In [1]:
import zipfile

def get_geospatial_whole_island(api_url, api_key, layer_id, folder_name='../datasets/geospatial_layer'):
    """
    Fetch data from the LTA DataMall API for geospatial whole island and save it as a CSV.
    
    Parameters:
    - api_url (str): The API endpoint URL for geospatial data.
    - api_key (str): Your LTA DataMall API key.
    - layer_id (str): Name of the geospatial layer (case sensitive).
    - folder_name (str): Folder where the extracted CSV will be saved. Defaults to 'geospatial_layer'.
    
    Returns:
    - str: Path to the downloaded CSV file.
    """
    headers = {
        'AccountKey': api_key,
        'accept': 'application/json'
    }

    params = {'ID': layer_id}
    
    response = requests.get(api_url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'value' in data:
            download_link = data['value'][0]['Link']  # Adjust index if there are multiple links
            
            # Download the zip file from the link
            zip_response = requests.get(download_link, stream=True)
            if zip_response.status_code == 200:
                # Ensure the folder exists
                os.makedirs(folder_name, exist_ok=True)
                
                # Define paths for zip and CSV file
                zip_file_path = os.path.join(folder_name, f'{layer_id}_geospatial.zip')
                
                # Save the zip file
                with open(zip_file_path, 'wb') as zip_file:
                    zip_file.write(zip_response.content)
                print(f"Zip file downloaded and saved to: {zip_file_path}")
                
                # Unzip the file
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(folder_name)
                
                # Identify the CSV file
                extracted_files = os.listdir(folder_name)
                csv_files = [file for file in extracted_files if file.endswith('.csv')]
                
                if csv_files:
                    csv_file_path = os.path.join(folder_name, csv_files[0])
                    print(f"CSV file extracted and saved to: {csv_file_path}")
                    
                    # Optionally remove the zip file after extraction
                    os.remove(zip_file_path)
                    
                    return csv_file_path
                else:
                    print("No CSV file found in the extracted files.")
                    return None
            else:
                print(f"Failed to download the zip file: {zip_response.status_code}")
                return None
        else:
            print("No 'value' key found in the response.")
            return None
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None
